# The model

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Define the model
class HandModel(nn.Module):
    def __init__(self):
        super(HandModel, self).__init__()
        self.fc1 = nn.Linear(63, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 4)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x = nn.functional.softmax(x, dim=0)
        return x

model = HandModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Import the data

In [9]:
import os
import pandas as pd
def getalldatafile():
    res=[]
    for file in os.listdir():
        if file.endswith(".txt") and file!="classes.txt":
            res.append(str(file.split(".")[0]))
    return res


df = pd.DataFrame(columns=['label'] + [f'd_{i:03}' for i in range(63)])
df.to_csv("data.csv", index=False,mode='w')

filenb=0
classes={}
for name in getalldatafile():
    classes[name]=filenb
    filenb+=1
    for line in open(name+".txt", "r").readlines():
        with open("data.csv", "a") as f:
            f.write(str(classes[name])+","+line.removesuffix("\n")+"\n")
    os.remove(name+".txt")
    print("file "+name+".txt done and removed")

# ajout fichier de classe
with open("classes.txt", "w") as f:
    for key in classes.keys():
        f.write(key+"\n")


file a.txt done and removed
file b.txt done and removed
file c.txt done and removed
file d.txt done and removed


# Init Dataset

In [10]:
# Load the data from the CSV file
data = pd.read_csv("data.csv")

# Separate the labels and the inputs
labels = data['label'].values
inputs = data.drop(columns=['label']).values

# Convert the inputs and labels to PyTorch tensors
inputs_tensor = torch.tensor(inputs, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

# Create a DataLoader
dataset = TensorDataset(inputs_tensor, labels_tensor)
train_ds,test_ds = torch.utils.data.random_split(dataset, [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))])
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=32, shuffle=True)

# Evaluation

In [11]:
def evaluate_accuracy():
    iclasses = {v: k for k, v in classes.items()}
    model.eval()
    correct = 0
    total = 0
    class_correct = {class_name: 0 for class_name in classes}
    class_total = {class_name: 0 for class_name in classes}
    
    with torch.no_grad():
        for inputs, labels in test_dl:
            outputs = model(inputs)
            _,predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            for label, prediction in zip(labels, predicted):
                if label == prediction:
                    class_correct[iclasses[label.item()]] += 1
                class_total[iclasses[label.item()]] += 1
    
    global_accuracy = correct / total
    class_accuracy = {class_name: class_correct[class_name] / class_total[class_name] for class_name in classes}
    
    print(f'Global Accuracy: {global_accuracy:.2f}')
    for class_name, accuracy in class_accuracy.items():
        print(f'Accuracy of class {class_name}: {accuracy:.2f}')


evaluate_accuracy()

Global Accuracy: 0.16
Accuracy of class a: 0.01
Accuracy of class b: 0.09
Accuracy of class c: 0.39
Accuracy of class d: 0.13



# TRAINING


In [12]:
evaluate_accuracy()
# Training loop
num_epochs = 400
for epoch in range(num_epochs):
    for inputs, labels in train_dl:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

evaluate_accuracy()

Global Accuracy: 0.18
Accuracy of class a: 0.00
Accuracy of class b: 0.08
Accuracy of class c: 0.39
Accuracy of class d: 0.20
Epoch 1/400, Loss: 1.2289178371429443
Epoch 2/400, Loss: 1.2300201654434204
Epoch 3/400, Loss: 1.228576898574829
Epoch 4/400, Loss: 1.2291069030761719
Epoch 5/400, Loss: 1.2272191047668457
Epoch 6/400, Loss: 1.2261853218078613
Epoch 7/400, Loss: 1.2280702590942383
Epoch 8/400, Loss: 1.233670949935913
Epoch 9/400, Loss: 1.252577781677246
Epoch 10/400, Loss: 1.2290358543395996
Epoch 11/400, Loss: 1.2251989841461182
Epoch 12/400, Loss: 1.226038932800293
Epoch 13/400, Loss: 1.2275432348251343
Epoch 14/400, Loss: 1.2256124019622803
Epoch 15/400, Loss: 1.2251579761505127
Epoch 16/400, Loss: 1.2252764701843262
Epoch 17/400, Loss: 1.2268853187561035
Epoch 18/400, Loss: 1.2263600826263428
Epoch 19/400, Loss: 1.2246915102005005
Epoch 20/400, Loss: 1.2259483337402344
Epoch 21/400, Loss: 1.2263940572738647
Epoch 22/400, Loss: 1.2279897928237915
Epoch 23/400, Loss: 1.2257598

# Save Model

In [13]:
#Save the model
torch.save(model.state_dict(), 'handmodel.pth')